In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Model, load_model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, AveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from imagenet_utils import preprocess_input

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_width, img_height = 224, 224
train_data_dir = "images/train_col224"
validation_data_dir = "images/dev_col224"
nb_train_samples = 32488
nb_validation_samples = 5723
batch_size = 32
epochs = 10

In [3]:
# load the original ResNet without top layer
model = applications.ResNet50(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [4]:
# adding custom Layers 
x = model.output
x = Flatten(name='flatten')(x)
out = Dense(18, activation="softmax", name='output_layer')(x)

# create the final model
model_final = Model(input=model.input, outputs=out)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ou..., inputs=Tensor("in...)`
  import sys


In [12]:
model_final.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [10]:
# freeze first 5 layers
for layer in model_final.layers[:37]:
    layer.trainable = False

In [6]:
def print_layer_trainable(model_name):
    for layer in model_name.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))

In [11]:
print_layer_trainable(model_final)

False:	input_1
False:	conv1
False:	bn_conv1
False:	activation_1
False:	max_pooling2d_1
False:	res2a_branch2a
False:	bn2a_branch2a
False:	activation_2
False:	res2a_branch2b
False:	bn2a_branch2b
False:	activation_3
False:	res2a_branch2c
False:	res2a_branch1
False:	bn2a_branch2c
False:	bn2a_branch1
False:	add_1
False:	activation_4
False:	res2b_branch2a
False:	bn2b_branch2a
False:	activation_5
False:	res2b_branch2b
False:	bn2b_branch2b
False:	activation_6
False:	res2b_branch2c
False:	bn2b_branch2c
False:	add_2
False:	activation_7
False:	res2c_branch2a
False:	bn2c_branch2a
False:	activation_8
False:	res2c_branch2b
False:	bn2c_branch2b
False:	activation_9
False:	res2c_branch2c
False:	bn2c_branch2c
False:	add_3
False:	activation_10
True:	res3a_branch2a
True:	bn3a_branch2a
True:	activation_11
True:	res3a_branch2b
True:	bn3a_branch2b
True:	activation_12
True:	res3a_branch2c
True:	res3a_branch1
True:	bn3a_branch2c
True:	bn3a_branch1
True:	add_4
True:	activation_13
True:	res3b_branch2a
True:	bn3b

In [ ]:
# load the saved model
# model_final = load_model('models/ResNet50_trained_f2c.h5')
# model_final.summary()

In [13]:
# compile the model
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])

In [14]:
# initiate the train and validation generators with data augumentation
train_datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        preprocessing_function=preprocess_input,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

# save the model according to the conditions
checkpoint = ModelCheckpoint("models/ResNet50_trained_f2c.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

Found 32488 images belonging to 18 classes.
Found 5723 images belonging to 18 classes.


In [15]:
print(train_generator.class_indices)
print(validation_generator.class_indices)

{'BabyBibs': 0, 'BabyHat': 1, 'BabyPants': 2, 'BabyShirt': 3, 'PackageFart': 4, 'womanshirtsleeve': 5, 'womencasualshoes': 6, 'womenchiffontop': 7, 'womendollshoes': 8, 'womenknittedtop': 9, 'womenlazyshoes': 10, 'womenlongsleevetop': 11, 'womenpeashoes': 12, 'womenplussizedtop': 13, 'womenpointedflatshoes': 14, 'womensleevelesstop': 15, 'womenstripedtop': 16, 'wrapsnslings': 17}
{'BabyBibs': 0, 'BabyHat': 1, 'BabyPants': 2, 'BabyShirt': 3, 'PackageFart': 4, 'womanshirtsleeve': 5, 'womencasualshoes': 6, 'womenchiffontop': 7, 'womendollshoes': 8, 'womenknittedtop': 9, 'womenlazyshoes': 10, 'womenlongsleevetop': 11, 'womenpeashoes': 12, 'womenplussizedtop': 13, 'womenpointedflatshoes': 14, 'womensleevelesstop': 15, 'womenstripedtop': 16, 'wrapsnslings': 17}


In [16]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    callbacks = [checkpoint])

Epoch 1/10
1015/1015 [==============================] - 19775s 19s/step - loss: 3.0861 - acc: 0.1146 - val_loss: 3.3286 - val_acc: 0.1452

Epoch 00001: val_acc improved from -inf to 0.14519, saving model to models/ResNet50_trained_f2a.h5
Epoch 2/10
1015/1015 [==============================] - 10871s 11s/step - loss: 1.8416 - acc: 0.3766 - val_loss: 2.3737 - val_acc: 0.3298

Epoch 00002: val_acc improved from 0.14519 to 0.32982, saving model to models/ResNet50_trained_f2a.h5
Epoch 3/10
1015/1015 [==============================] - 10275s 10s/step - loss: 1.2736 - acc: 0.5631 - val_loss: 1.9271 - val_acc: 0.4214

Epoch 00003: val_acc improved from 0.32982 to 0.42137, saving model to models/ResNet50_trained_f2a.h5
Epoch 4/10
1015/1015 [==============================] - 10261s 10s/step - loss: 1.0214 - acc: 0.6478 - val_loss: 1.6409 - val_acc: 0.4825

Epoch 00004: val_acc improved from 0.42137 to 0.48252, saving model to models/ResNet50_trained_f2a.h5
Epoch 5/10
1015/1015 [=================

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model

# plot_model(model, to_file='model.png')
SVG(model_to_dot(model_final).create(prog='dot', format='svg'))